In [1]:
 !pip  install datasets
 !pip  install  transformers
 !pip  install nlp
 !pip  install SentencePiece 
 !pip install rouge-score
!pip install Rouge

     |████████████████████████████████| 194kB 13.1MB/s 
     |████████████████████████████████| 112kB 22.9MB/s 
     |████████████████████████████████| 245kB 26.1MB/s 
     |████████████████████████████████| 2.0MB 12.3MB/s 
     |████████████████████████████████| 890kB 42.3MB/s 
     |████████████████████████████████| 3.2MB 49.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=6dd6f0e1d1437d5759a68897764ca40051d25cd97630eccca71099da156da15a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 1.7MB 13.5MB/s 
     |████████████████████████████████| 1.2MB 12.2MB/s 


In [2]:
# standard imports
import csv
import os
import re
import os
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
#import torch
import tensorflow as tf
import nlp
#import sentencepiece
from rouge_score import rouge_scorer
%matplotlib inline 

In [ ]:
from transformers import TFT5Model, T5Tokenizer  , TFT5ForConditionalGeneration, TFTrainer, TFTrainingArguments

In [ ]:
from rouge import Rouge
rouge = nlp.load_metric("rouge")

In [4]:
#confirm GPU is active
tf.test.gpu_device_name()

''

In [ ]:
#extract clean files
test_df = pd.read_csv('drive/MyDrive/test_clean.csv' ).drop('Unnamed: 0', axis=1)
train_df = pd.read_csv('drive/MyDrive/train_clean.csv' ).drop('Unnamed: 0', axis=1)


In [ ]:
print(train_df.shape)
print(test_df.shape)

(18949, 6)
(3269, 6)


In [ ]:
example_text = train_df.text[:50]
example_summ = train_df.summary[:50]

In [ ]:
#pull model
t5_model = 't5-small'
t5_tokenizer = T5Tokenizer.from_pretrained(t5_model)
t5 = TFT5ForConditionalGeneration.from_pretrained(t5_model)

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
#limit db to shorter text lengths
def shorten(df , max_text_length = 4000, split_length = 1000):
  df = df[df.text.apply(len) <= max_text_length].loc[: , ['text' , 'summary']]
  count = df.shape[0]
  tmp = pd.DataFrame(data=None, columns=['text','id'])
  for i in range(count):
    txt = df.text.iloc[i]
    for j in range(0,len(txt),split_length):
      tmp_txt = txt[j:j+split_length]
      tmp1 = pd.DataFrame(["summarize: " + tmp_txt , i] , index=['text','id']).T
      tmp = pd.concat([tmp,tmp1])

    ids = [i for i in range(df.shape[0])]
    tmp_summary = df.summary 
    tmp_summary = pd.DataFrame(tmp_summary)
    tmp_summary['id'] = ids

  return tmp , tmp_summary


In [ ]:
#create function to recompile sentences
def compile_sents(df , sent_list):
  output= pd.DataFrame(data=None, columns=['comp_sent','id'])

  tmp = pd.DataFrame( zip(df.id , sent_list), columns=['id' , 'text'])
  for i in set(tmp.id):
    txt = tmp.text[tmp.id == i].str.cat(sep=" ")
    
    tmp1 = None
    for word in ['<pad>'  , '   ', '<unk>' , 'SECTION-HEADER>',  '<extra_id_0>' ,'<extra_id_1>' , '<extra_id_2>' , '<extra_id_3>' , ',,' , '..' , '</s> ', 'summarize: ' , 'SHORT TITLE' , ': . ']:
      txt =  txt.replace(word, '')
    
    txt = txt.replace('  ',' ')
    
    tmp1 = pd.DataFrame([txt ,  i] ).T
    tmp1.columns = ['comp_sent','id']
    output = pd.concat([output, tmp1])

  return output

In [ ]:
#score output
def return_rouge(pred_sents , true_sents):
  rouge = Rouge()
  output = rouge.get_scores(pred_sents, true_sents, avg=True)
  return output 

In [ ]:
#create model function -> outputs rouge scores
def model_output(df_train , df_summ,tokenizer,model, max_length=200 , early_stopping=False ,  num_beams=1):

    #compile summaries
    filt = df_summ.id.apply( lambda x: x in list(set(df_train.id) ))
    train_summ = df_summ[filt]

    #inputs
    inputs = tokenizer(list(df_train.text), return_tensors='tf' , max_length=max_length ,  padding=True, truncation=True).input_ids

    #run model
    outputs =  model.generate(inputs, min_length=20 , max_length=100,early_stopping=early_stopping, num_beams=num_beams)

    #convert to words
    output_sents = [tokenizer.decode(i) for i in outputs.numpy()]

    #compile sentences
    comp_sents = compile_sents(df_train , output_sents) 

    #calc rouge
    #rouge_outputs = return_rouge(comp_sents , list(train_summ.summary))
    return comp_sents

In [ ]:
def  run_model( df_train , df_summ ,tokenizer,model , block_size = 50, max_length=200, early_stopping=False ,  num_beams=1):
    output= pd.DataFrame(data=None, columns=['comp_sent','id'])

    step = 0
    for i in range(0,df_train.shape[0]  , block_size ):
        train_txt = df_train[i:i+block_size]

        outs = model_output(train_txt, df_summ,tokenizer,model, max_length=max_length , early_stopping=early_stopping, num_beams=num_beams)
        output = pd.concat([output , outs])

        step+=1
        if step%25 ==0:
            print(step * block_size , " counts done")
    
    compiled = pd.merge(output , df_summ , left_on= 'id' , right_on='id')

    compiled = compiled[compiled.summary.duplicated()==False]

    preds , true = list(compiled.comp_sent) ,   list(compiled.summary)
    return return_rouge(preds , true) , preds , true

In [ ]:
#make dbs
#train_size = 3000
#short_train , short_train_summ = shorten(train_df[:train_size]  , max_text_length = 15000, split_length=1000)
short_test , short_test_summ = shorten(test_df  , max_text_length = 15000, split_length=1000)
ex_train , ex_summ = shorten(test_df[:50]  , max_text_length = 150000, split_length=1000)
#print(short_train.shape)
print('example set shape:',ex_train.shape)
print("test set shape:",short_test.shape)

example set shape: (411, 2)
test set shape: (26079, 2)


In [ ]:
#print results to csv
def output_to_csv(outputs,filename= "drive/MyDrive/base_T5_small_output.csv"):
    csv_columns = ['rouge','f','p','r']
    dict_data =outputs

    csv_file =filename
    try:
        os.remove(csv_file)
    except: 
        pass

    try:
        with open(csv_file, 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
            writer.writeheader()
            for key in dict_data.keys():
                tmp = dict_data[key]
                tmp['rouge'] = key
                writer.writerow(tmp)
            print("Loaded File: {}".format(filename))

    except IOError:
        print("I/O error")

In [ ]:
 ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ##
 ## t5 small model on test  - no beam search
  ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ##

In [ ]:
#run the model
outs  , preds , true = run_model(short_test , short_test_summ,t5_tokenizer,t5 )

1250  counts done
2500  counts done
3750  counts done
5000  counts done
6250  counts done
7500  counts done
8750  counts done
10000  counts done
11250  counts done
12500  counts done
13750  counts done
15000  counts done
16250  counts done
17500  counts done
18750  counts done
20000  counts done
21250  counts done
22500  counts done
23750  counts done
25000  counts done


In [ ]:
output_to_csv(outputs=outs,filename= "drive/MyDrive/base_T5_small_output_test.csv")
pd.DataFrame([preds,true],index=['predicted','true_y']).T.to_csv("drive/MyDrive/base_T5_small_preds_true_test.csv")

Loaded File: drive/MyDrive/base_T5_small_output_test.csv


In [ ]:
 ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ##
 ## t5 base model on test 
  ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ##

In [ ]:
#make dbs
short_test , short_test_summ = shorten(test_df  , max_text_length = 15000, split_length=1500)
print(short_test.shape)

(17942, 2)


In [ ]:
#load base model
t5_model = 't5-base'
t5_tokenizer = T5Tokenizer.from_pretrained(t5_model)
t5 = TFT5ForConditionalGeneration.from_pretrained(t5_model)

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
#run the model
outs  , preds , true = run_model(short_test , short_test_summ,t5_tokenizer,t5 ,max_length=300)
output_to_csv(outputs=outs,filename= "drive/MyDrive/base_T5_base_output_test.csv")
pd.DataFrame([preds,true],index=['predicted','true_y']).T.to_csv("drive/MyDrive/base_T5_base_preds_true_test.csv")

1250  counts done
2500  counts done
3750  counts done
5000  counts done
6250  counts done
7500  counts done
8750  counts done
10000  counts done
11250  counts done
12500  counts done
13750  counts done
15000  counts done
16250  counts done
17500  counts done
Loaded File: drive/MyDrive/base_T5_base_output_test.csv


In [ ]:
 ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ##
 ## t5 small model on test  - 5 beam search
  ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ##

In [ ]:
#pull model
t5_model = 't5-small'
t5_tokenizer = T5Tokenizer.from_pretrained(t5_model)
t5 = TFT5ForConditionalGeneration.from_pretrained(t5_model)

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
#make dbs
short_test , short_test_summ = shorten(test_df  , max_text_length = 15000, split_length=1200)
print(short_test.shape)

(22026, 2)


In [ ]:
#run the model
outs  , preds , true = run_model(short_test , short_test_summ,t5_tokenizer,t5 ,max_length=200,early_stopping=True, num_beams=5)
output_to_csv(outputs=outs,filename= "drive/MyDrive/base_T5_small_5beam_output_test.csv")
pd.DataFrame([preds,true],index=['predicted','true_y']).T.to_csv("drive/MyDrive/base_T5_small_5beam_preds_true_test.csv")

1250  counts done
2500  counts done
3750  counts done
5000  counts done
6250  counts done
7500  counts done
8750  counts done
10000  counts done
11250  counts done
12500  counts done
13750  counts done
15000  counts done
16250  counts done
17500  counts done
18750  counts done
20000  counts done
21250  counts done
Loaded File: drive/MyDrive/base_T5_small_5beam_output_test.csv


In [ ]:
 ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ##
 ##experiments
  ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ##

In [ ]:
#pull model
t5_model = 't5-small'
tokenizer = T5Tokenizer.from_pretrained(t5_model)
t5 = TFT5ForConditionalGeneration.from_pretrained(t5_model)

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
#filter down long text threads for model
def convert_long_text(text):
    ddd = re.split('<SECTION-HEADER> ',text)
    try:
        input_tokens = tokenizer(['summarize: '+ i for i in ddd[2:]],return_tensors='tf',max_length=512,padding=True).input_ids
    except:
        input_tokens = tokenizer(['summarize: '+ i for i in ddd],return_tensors='tf',max_length=512,padding=True).input_ids
    outs = t5.generate(input_ids=input_tokens,min_length=50,max_length=100 , early_stopping=True, num_beams=2,temperature=7)
    ooo = [tokenizer.decode(i) for i in outs.numpy()]
    cc = ''.join(ooo).replace('<pad>','').replace('  ','').replace('</s>','')
    return '<pad> summarize:' + cc

#create mask layers
def create_mask(x):
    max_=512
    mask = tokenizer(x,return_tensors='tf',max_length=max_,truncation='longest_first',padding=True).attention_mask.numpy()[0]
    if mask.shape[0]<max_:
        adds = max_ - mask.shape[0]
        mask = np.concatenate((mask , adds* [0]))

    return mask

#create eoncoding layers
def create_encoding(x):
    max_=512
    encod = tokenizer(x,return_tensors='tf',max_length=max_,truncation='longest_first',padding=True).input_ids.numpy()[0]
    if encod.shape[0]<max_:
        adds = max_ - encod.shape[0]
        encod= np.concatenate((encod , adds* [0]))

    return encod

In [ ]:
#make shorter train df
train = train_df[:2000]
# abbrev=[]
# for i in train.text:
#     abbrev.append(convert_long_text(i))

In [ ]:
full_data = pd.DataFrame([pd.read_csv('drive/MyDrive/2k_t5_summ') . t5_summ,train.summary]).T

In [ ]:
max_tensor_length=512

train_text_tensors = tokenizer(list( full_data[:1500].t5_summ) , return_tensors='tf' , max_length=max_tensor_length , padding=True,truncation=True) 
train_summ_tensors = tokenizer(list( full_data[:1500].summary) , return_tensors='tf' , max_length=max_tensor_length , padding=True,truncation=True)
val_text_tensors = tokenizer(list( full_data[1500:].t5_summ) , return_tensors='tf' , max_length=max_tensor_length , padding=True,truncation=True) 
val_summ_tensors = tokenizer(list( full_data[1500:].summary), return_tensors='tf' , max_length=max_tensor_length , padding=True,truncation=True)

In [ ]:
data_sets = [train_text_tensors ,  train_summ_tensors,val_text_tensors,val_summ_tensors]

for i in range(len(data_sets)-1):
    print(data_sets[i].input_ids.numpy().shape[1] == data_sets[i+1].input_ids.numpy().shape[1])

True
True
True


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_text_tensors),train_summ_tensors.input_ids))
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_text_tensors),val_summ_tensors.input_ids))

In [ ]:
 def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rougeL"])["rougeL"].mid

    return {
        "rougeL_precision": round(rouge_output.precision, 4),
        "rougeL_recall": round(rouge_output.recall, 4),
        "rougeL_fmeasure": round(rouge_output.fmeasure, 4),
    }


#train model
training_args = TFTrainingArguments(
    output_dir='drive/MyDrive/results',           # output directory
    num_train_epochs=5,              # total # of training epochs
    per_device_train_batch_size=2,  # batch size per device during training
    per_device_eval_batch_size=1,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='drive/MyDrive/logs',            # directory for storing logs
    #evaluate_during_training=True,
    do_train=True,
    do_eval=True,
    # fp16=True,  # This has a known bug with t5
    gradient_accumulation_steps=16,
    logging_steps=100,
    prediction_loss_only=True,
    eval_steps=100,
    evaluation_strategy ="epoch" ,
    overwrite_output_dir=True,
    save_total_limit=10
)



In [ ]:
model=None
trainer_eval=None
with training_args.strategy.scope():
    model = TFT5ForConditionalGeneration.from_pretrained('t5-small')

    trainer = TFTrainer(
        model=model,                         # the instantiated 🤗 Transformers model to be trained
        args=training_args,                  # training arguments, defined above
        compute_metrics=compute_metrics,
        train_dataset=train_dataset,    # tensorflow_datasets training dataset
        eval_dataset=val_dataset       # tensorflow_datasets evaluation dataset
    )

    trainer.train()
    trainer.save_model('drive/MyDrive/model_t5_small_2k_train')
    #trainer_pred = trainer.predict(val_dataset)


All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
#pull trained model
tokenizer = T5Tokenizer.from_pretrained(t5_model)
t5_custom = TFT5ForConditionalGeneration.from_pretrained('drive/MyDrive/model_t5_small_2k_train')

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at drive/MyDrive/model_t5_small_2k_train.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
input_tokens = tokenizer('summarize: '+ test_df.text[10] ,return_tensors='tf',max_length=512,padding=True).input_ids
cc = t5_custom.generate(input_ids=input_tokens,min_length=50,max_length=100 , early_stopping=True, num_beams=2,temperature=7)

In [ ]:
results = trainer.evaluate()

In [ ]:
inputs = tokenizer(['summarize: '+ i for i in  test_df.text[10:12]] ,return_tensors='tf',max_length=512,padding=True,truncation=True)#.input_ids
labels = tokenizer(['summarize: '+ i for i in  test_df.summary[10:12]] ,return_tensors='tf',max_length=512,padding=True,truncation=True).input_ids
inputs = inputs[tf.newaxis]

mask_adds = 512 - labels.numpy().shape[1]
labels = np.concatenate((labels.numpy()[0]  , mask_adds* [0]))
labels = tf.convert_to_tensor(labels)

t5_custom(input_ids=inputs , labels = labels)

KeyError: ignored

In [ ]:
inputs.keys()#.numpy().shape

dict_keys(['input_ids', 'attention_mask'])